In [236]:
import os
from configparser import ConfigParser
import pandas as pd

# 1. Configure SAP NetWeaver SDK

In [237]:
proj_dir = os.getcwd()

In [238]:
os.environ['SAPNWRFC_HOME'] = proj_dir + '\\nwrfcsdk' 

In [239]:
from pyrfc import Connection, ABAPRuntimeError

# 2. Locad configuration

In [240]:
configur = ConfigParser()

In [241]:
configur.read('config.ini')

['config.ini']

In [242]:
ASHOST=configur.get('SAP','host')
CLIENT=configur.get('SAP','client')
SYSNR=configur.get('SAP','instance')
USER=configur.get('SAP','username')
PASSWD=configur.get('SAP','password')

# 3. Connect to SAP

In [243]:
conn = Connection(
    ashost=ASHOST, 
    sysnr=SYSNR, 
    client=CLIENT, 
    user=USER, 
    passwd=PASSWD,
    lang="EN"
    )

In [244]:
def conver_results_to_object(result):
    results = []
    for row in result['DATA']:
        
            d = row['WA'].split('|')
            fields = result['FIELDS']
            o = {}
            for i,v in enumerate(d):
                o[fields[i]['FIELDNAME']] = v
            results.append(o)
    return results

# 4. Define Functions

## 4.1 Get PO Invoices

In [245]:
def get_po_invoices(conn,po):
    po_results = []
    kwargs = dict(
                QUERY_TABLE = 'EKBE',
                DELIMITER = '|',
                NO_DATA = '',
                FIELDS = [ 
                    {'FIELDNAME':'EBELN'} ,
                    {'FIELDNAME':'EBELP'} ,
                    {'FIELDNAME':'ZEKKN'} ,
                    {'FIELDNAME':'VGABE'} ,
                    {'FIELDNAME':'GJAHR'} ,
                    {'FIELDNAME':'BELNR'} ,
                    {'FIELDNAME':'BUZEI'} ,
                        ],
                ROWSKIPS = 0 ,
                ROWCOUNT = 500,
                OPTIONS = [
                    "MANDT = '"+ CLIENT +"'", 
                    " AND EBELN = '"+ po +"'",
                    " AND VGABE = '2'"
                    ]
            )
    try:
        result = conn.call('RFC_READ_TABLE',**kwargs)
        po_results = conver_results_to_object(result)
    except ABAPRuntimeError as error:
        print(error.message)
    
    return po_results

## 4.2 Get FI Document from PO Invoice

In [246]:
def get_fi_doc(conn,ObjectKey):
    po_results = []
    kwargs = dict(
                QUERY_TABLE = 'BKPF',
                DELIMITER = '|',
                NO_DATA = '',
                FIELDS = [ 
                    {'FIELDNAME':'BUKRS'} ,
                    {'FIELDNAME':'BELNR'} ,
                    {'FIELDNAME':'GJAHR'} ,
                    {'FIELDNAME':'BLART'} ,
                    {'FIELDNAME':'AWTYP'} ,
                    {'FIELDNAME':'AWKEY'} ,
                        ],
                ROWSKIPS = 0 ,
                ROWCOUNT = 500,
                OPTIONS = [
                    "MANDT = '"+ CLIENT +"'", 
                    " AND AWKEY = '"+ ObjectKey +"'",
                    ]
            )
    try:
        result = conn.call('RFC_READ_TABLE',**kwargs)
        po_results = conver_results_to_object(result)
    except ABAPRuntimeError as error:
        print(error.message)
    
    return po_results

## 4.3 Get FI Document items

In [247]:
def get_fi_doc_items(conn,BUKRS,BELNR,GJAHR):
    results = []
    kwargs = dict(
                QUERY_TABLE = 'BSEG',
                DELIMITER = '|',
                NO_DATA = '',
                FIELDS = [ 
                    {'FIELDNAME':'BUKRS'} ,
                    {'FIELDNAME':'BELNR'} ,
                    {'FIELDNAME':'GJAHR'} ,
                    {'FIELDNAME':'BUZEI'} ,
                    {'FIELDNAME':'AUGDT'} ,
                    {'FIELDNAME':'AUGCP'} ,
                    {'FIELDNAME':'AUGBL'} ,
                    {'FIELDNAME':'BSCHL'} ,
                        ],
                ROWSKIPS = 0 ,
                ROWCOUNT = 500,
                OPTIONS = [
                    "MANDT = '"+ CLIENT +"'", 
                    " AND BUKRS = '"+ BUKRS +"'",
                    " AND BELNR = '"+ BELNR +"'",
                    " AND GJAHR = '"+ GJAHR +"'",
                    " AND BSCHL = '31'",
                    ]
            )
    try:
        result = conn.call('RFC_READ_TABLE',**kwargs)
        results = conver_results_to_object(result)
    except ABAPRuntimeError as error:
        print(error.message)
    
    return results

# 5. Read PO Invoices

In [248]:
#po_data = get_po_invoices(conn,'4500019671')

In [249]:
po_list = ['4500019671','4500019602','4500019670','3005000125','4500019559']

In [250]:
po_data = []
for po in po_list:
    result = get_po_invoices(conn,po)
    po_data += result

In [251]:
df_po_ri = pd.DataFrame(data=po_data)

In [252]:
df_po_ri['AWKEY'] = df_po_ri['BELNR'] + df_po_ri['GJAHR']

In [253]:
AWKEY_docs = list(df_po_ri['AWKEY'].drop_duplicates())

# 6. Retriave FI documents 

In [254]:
fi_doc_hd = []
for objKey in AWKEY_docs:
    result = get_fi_doc(conn, objKey)
    fi_doc_hd += result

In [255]:
df_fi_doc_hd = pd.DataFrame(data=fi_doc_hd)

In [256]:
fi_doc_it = []
for fi_doc in fi_doc_hd:
    result = get_fi_doc_items(conn, fi_doc['BUKRS'],fi_doc['BELNR'],fi_doc['GJAHR'])
    fi_doc_it += result

In [257]:
df_fi_doc_it = pd.DataFrame(data=fi_doc_it)

In [258]:
df_fi_doc_it.loc[ df_fi_doc_it['AUGDT'] == '00000000' ,'AUGDT'] = ''

In [259]:
df_fi_doc_it.loc[ df_fi_doc_it['AUGCP'] == '00000000' ,'AUGCP'] = ''

In [260]:
df_fi_doc = pd.merge(df_fi_doc_hd,df_fi_doc_it,on=['BUKRS','BELNR','GJAHR'])

In [261]:
df_1 = pd.merge(df_po_ri,df_fi_doc,on='AWKEY')

In [262]:
df_1[['EBELN','GJAHR_x','BELNR_x','BELNR_y','GJAHR_y','AUGDT','AUGCP','AUGBL']]

,EBELN,GJAHR_x,BELNR_x,BELNR_y,GJAHR_y,AUGDT,AUGCP,AUGBL
0,4500019671,2025,5105609642,5100000000,2025,,,
1,4500019671,2025,5105609643,5100000001,2025,,,
2,4500019671,2025,5105609644,5100000002,2025,20250816,20250816,1500000000
3,4500019671,2025,5105609645,5100000003,2025,,,
4,4500019671,2025,5105609643,5100000001,2025,,,
5,3005000125,2003,5105607042,5100000509,2003,,,
6,3005000125,2003,5105607042,5100000509,2003,,,
7,3005000125,2003,5105607042,5100000509,2003,,,
8,4500019559,2017,5105609633,5100000001,2017,,,
